<a href="https://colab.research.google.com/github/sudeepan/Symbolica-Colab/blob/main/Symbolica_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="left"><img src="https://symbolica.io/icon.svg" width="150px"></p>

## Setup

In [ ]:
!pip install symbolica
from IPython.display import Markdown as md
from symbolica import set_license_key

# do not use this key in your own setup
# rather: get a license easily at https://symbolica.io/docs/get_started.html#license
set_license_key('gcolab-demo-key-do-not-use-elsewhere')

## Introduction


Symbolica is a new computer algebra system that aims to handle expressions with billions of terms, while at the same time being easy to use. It has APIs for Python, Rust and C++.

In this Jupyter notebook we will explore some features of Symbolica.

--------------

Declare a variable and a function

In [ ]:
from symbolica import Expression

x = Expression.var('x')
f = Expression.fun('f')

Create a new expression by composing variables, functions, and numbers

In [ ]:
e = f(x) + 4
print(e)

f(x)+4


Expressions can also be parsed, with liberal parsing rules

In [ ]:
e = Expression.parse('f(x,y)+ 5 (x + y)^2 + 1_123_456')
print(e)

5*(x+y)^2+f(x,y)+1123456


The output format can be easily changed

In [ ]:
md(e.to_latex())

$$5 (x+y)^{2}+f(x,y)+1123456$$

## Pattern matching

Replace all occurrences of $x$ in $x f(x, 1, y^x)$ by 5

In [ ]:
e = Expression.parse('x*f(x,1,y^x)')
e = e.replace_all(x, 5)
md(e.to_latex())

$$5 f(5,1,y^{5})$$

Use a wildcard, a variable ending in an underscore `_` to match to any subexpression

In [ ]:
x_ = Expression.var('x_')

e = f(5).replace_all(f(x_), f(x_ + 1))
md(e.to_latex())

$$f(6)$$

Repeated wildcards must match to the same subexpressions

In [ ]:
y_, z_, w_ = Expression.vars('y_', 'z_', 'w_')

e = f(1,2,3,2,4).replace_all(f(x_,y_,z_,y_,w_), f(x_,z_,w_))
md(e.to_latex())

$$f(1,3,4)$$

Wildcards can match to multiple ranges / subexpressions

In [ ]:
e = f(1,2,3,4).replace_all(f(x_,4), f(x_))
md(e.to_latex())

$$f(1,2,3)$$

Conditions can be set on wildcards

In [ ]:
e = (f(1,4) + f(1,2,3,4)).replace_all(f(x_,4), f(x_), x_.req_len(2, None)) # minimum length is 2
md(e.to_latex())

$$f(1,4)+f(1,2,3)$$

Conditions can be set by user-defined functions

In [ ]:
e = (f(1)*f(2)*f(3)).replace_all(f(x_), 1, x_.req(lambda m: m == 2 or m == 3))
md(e.to_latex())

$$f(1)$$

Matches can be iterated over, like the `regex` module in Python

In [ ]:
from symbolica import Expression

x, y, z, n_, x_ = Expression.vars('x', 'y', 'z', 'n_', 'x_')
e = x**2*(y+z) + x**3*(y+z**2)

# match returns a dictionary that maps every wildcard to its value
coeff_list = [(m[n_], m[x_]) for m in e.match(x**n_*x_)]
for (pow, content) in coeff_list:
    print(x**pow, content)

x^2 y+z
x^3 y+z^2


### Transformers

The right-hand side can be used to apply arbitrarily complex operations after the wildcard substitution. For example:

In [ ]:
e = f((x+1)**2).replace_all(f(x_), f(x_.transform().expand()))
md(e.to_latex())


$$f(2 x+x^{2}+1)$$

Custom functions can be used as well

In [ ]:
e = f(2).replace_all(f(x_), x_.transform().map(lambda r: r**2))
md(e.to_latex())

$$4$$

Below is a more complicated example that computes the color of the triangle-box-triangle in QCD

In [ ]:
from symbolica import Expression

nc, i, x_,y_,z_, w_, a_ = Expression.vars('nc', 'i', 'x_', 'y_', 'z_', 'w_', 'a_')
T, f, trace = Expression.funs('T', 'f', 'trace')

# tri-box-tri
e = T(1,12)*T(12,11)*T(11,13,10)*T(10,9)*T(9,15,8)*T(8,7)*T(7,6)*T(6,5)*T(5,15,4)*T(4,3)*T(3,13,2)*T(2, 1)

# collect the color string into a trace
last_e = T()
while last_e != e:
    last_e = e
    e = e.replace_all(T(x_,z_)*T(z_, w_), T(x_,w_), z_.req_len(1,1))

e = e.replace_all(T(x_,y_,x_), trace(y_), x_.req_len(1,1))

# requirements for the patterns below
req = x_.req_len(0,None) & y_.req_len(1,1) & z_.req_len(0,None) & w_.req_len(0,None)

last_e = T()
while last_e != e:
    last_e = e
    e = e.replace_all(trace(), nc)
    e = e.replace_all(trace(x_), 0, x_.req_len(1,1))
    e = e.replace_all(trace(x_,y_,z_,y_,w_), trace(x_,w_)*trace(z_) / 2 - trace(x_,z_,w_) / 2 / nc, req)
    e = e.replace_all(f(x_,y_,z_), -2 * i * trace(x_,y_,z_) + 2 * i * trace(z_,y_,x_))
    e = e.replace_all(trace(x_,y_,z_) * trace(w_,y_,a_), trace(x_,a_,w_,z_) / 2  - trace(x_,z_) * trace(w_,a_) / 2 / nc, req)

print(e.expand())
print(e.replace_all(nc, 3))
md('$$' + e.to_rational_polynomial().to_latex() + '$$')

-1/2*nc+1/4*nc^-1+1/4*nc^3
16/3


$$\frac{1-2nc^{2}+nc^{4}}{4nc}$$

### Programmable patterns

Python can be used to make programmable patterns. In the following example we will apply it to compute a trace using the following simplification algoritm:


$$Tr(\mu_1,\ldots,\mu_n) = \sum_{k=2}^n (-1)^k g(\mu_1,\mu_k) Tr(\mu_2,\ldots,\mu_{k-1},\mu_{k+1},\ldots \mu_n)$$

In [ ]:
from symbolica import Expression

x_ = Expression.var('x_')
p = Expression.vars(*['p' + str(i + 1) for i in range(10)])
p_ = Expression.vars(*['p' + str(i + 1) + '_' for i in range(10)])
f = Expression.fun('f')
d = Expression.fun('d', is_symmetric=True)

e = f(*p[:6])

# find the maximum chain length
max_length = 0
for m in e.match(f(x_)):
    max_length = max(max_length, len(m[x_]))

for l in range(max_length,0,-1):
    if l % 2 == 1:
        e = e.replace_all(f(*p_[:l]), 0) # odd trace is 0
    else:
        e = e.replace_all(f(*p_[:l]), sum((-1)**(k+1) * d(p_[0], p_[k]) * f(*p_[1:k], *p_[k+1:l]) for k in range(1,l))).expand()
e = e.replace_all(f(), 4)
print('\t ' + e.pretty_str(terms_on_new_line=True))

	 4*d(p1,p2)*d(p3,p4)*d(p5,p6)
	-4*d(p1,p2)*d(p3,p5)*d(p4,p6)
	+4*d(p1,p2)*d(p3,p6)*d(p4,p5)
	-4*d(p1,p3)*d(p2,p4)*d(p5,p6)
	+4*d(p1,p3)*d(p2,p5)*d(p4,p6)
	-4*d(p1,p3)*d(p2,p6)*d(p4,p5)
	+4*d(p1,p4)*d(p2,p3)*d(p5,p6)
	-4*d(p1,p4)*d(p2,p5)*d(p3,p6)
	+4*d(p1,p4)*d(p2,p6)*d(p3,p5)
	-4*d(p1,p5)*d(p2,p3)*d(p4,p6)
	+4*d(p1,p5)*d(p2,p4)*d(p3,p6)
	-4*d(p1,p5)*d(p2,p6)*d(p3,p4)
	+4*d(p1,p6)*d(p2,p3)*d(p4,p5)
	-4*d(p1,p6)*d(p2,p4)*d(p3,p5)
	+4*d(p1,p6)*d(p2,p5)*d(p3,p4)


## Features

Derivatives

In [ ]:
y = Expression.var('y')
e = 1/(x*y+1)
e = e.derivative(x)
md(e.to_latex())

$$-y (x y+1)^{-2}$$

Symbolica can process a huge expression one term at a time, in parallel. This means that the expression does not have to be in memory (similar to FORM).

In [ ]:
from symbolica import Transformer

e = ((1+x)**10).expand()
r = e.map(Transformer().replace_all(x, 6))
print(r)

282475249


## Polynomials

Symbolic has world-class rational polynomial arithmetic. Polynomials can easily be constructed:

In [ ]:
from symbolica import Expression, Polynomial
x, y = Expression.vars('x', 'y')
e = ((x + y + x**2)**3).expand()
p = e.to_polynomial() + Polynomial.parse('x^2*y + 1 + 2*x + y^4', ['x', 'y'])
md('$$' + p.to_latex() + '$$')

$$1+y^{3}+y^{4}+2x+3xy^{2}+4x^{2}y+3x^{2}y^{2}+x^{3}+6x^{3}y+3x^{4}+3x^{4}y+3x^{5}+x^{6}$$

Rational polynomial arithmetic is supported too:

In [ ]:
from symbolica import RationalPolynomial
p = RationalPolynomial.parse('1/x+(1+x)^2/(1+x+y)', ['x', 'y'])
md('$$' + p.to_latex() + '$$')

$$\frac{1+y+2x+2x^{2}+x^{3}}{x+xy+x^{2}}$$

Computing the greatest common divisor of polynomials is an expensive operation. Below Symbolica does it in a few seconds, whereas other computer algebra packages take more than a minute.

In [ ]:
a = Expression.parse('(1 + 3*x1 + 5*x2 + 7*x3 + 9*x4 + 11*x5 + 13*x6 + 15*x7)^7 - 1').to_rational_polynomial()
b = Expression.parse('(1 - 3*x1 - 5*x2 - 7*x3 + 9*x4 - 11*x5 - 13*x6 + 15*x7)^7 + 1').to_rational_polynomial()
g = Expression.parse('(1 + 3*x1 + 5*x2 + 7*x3 + 9*x4 + 11*x5 + 13*x6 - 15*x7)^7 + 3').to_rational_polynomial()
ag = a * g
bg = b * g
gcd = ag.gcd(bg) - g
print(gcd)

0


### Fast evaluation

Symbolica can write a polynomial in an optimized form that allows for fast evaluation. It can even produce C++ output that evaluates the polynomial.

In [ ]:
from symbolica import Expression
x, y = Expression.vars('x', 'y')
e = ((x + y + x**2)**3).expand().to_polynomial()
eval = e.optimize() # to_file='test2.cpp'

print(eval.evaluate([[1., 2.], [3. ,4.]]))

[64.0, 4096.0]


If chosen to generate a file, it produces output such as:

```cpp
#include <iostream>

template<typename T>
T evaluate(T x,T y) {
	T Z0,Z1,Z2,Z3,Z4,Z5,Z6,Z7;
	Z0 = x;
	Z1 = y;
	Z2 = T(3);
	Z3 = T(1);
	Z4 = T(6);
	Z5 = Z0+Z2;
	Z5 = Z0*Z5;
	Z6 = Z1+Z3;
	Z6 = Z2*Z6;
	Z5 = Z5+Z6;
	Z5 = Z0*Z5;
	Z7 = Z1*Z4;
	Z5 = Z3+Z5+Z7;
	Z5 = Z0*Z5;
	Z6 = Z1*Z6;
	Z5 = Z5+Z6;
	Z5 = Z0*Z5;
	Z6 = Z1*Z1;
	Z7 = Z2*Z6;
	Z5 = Z5+Z7;
	Z5 = Z0*Z5;
	Z6 = Z1*Z6;
	Z5 = Z5+Z6;
	return Z5;
}

int main() {
	std::cout << evaluate<double>(0.25,0.5) << std::endl;
}
```

## Numerical integration

Symbolica uses the Vegas algorithm to numerically integrate in a continuous grid

In [ ]:
from symbolica import NumericalIntegrator

def integrand(samples):
    res = []
    for sample in samples:
        res.append(sample.c[0]**2+sample.c[1]**2)
    return res

avg, err, chi_sq = NumericalIntegrator.continuous(2).integrate(integrand, min_error = 0.0007)

### Multi-channeling

Symbolica supports multi-channeling, by allowing to nest discrete layers and continuous grids.

In [ ]:
from symbolica import NumericalIntegrator, Sample
from math import sin, cos
def integrand(samples):
    res = []
    for sample in samples:
        if sample.d[0] == 0:
            res.append(sin(sample.c[0]))
        else:
            res.append(cos(sample.c[0]))
    return res

integrator = NumericalIntegrator.discrete(
    [NumericalIntegrator.continuous(1), NumericalIntegrator.continuous(1)])
avg, err, chi_sq = integrator.integrate(integrand, min_error = 0.003)

### More control

You can also do the sample-update loop yourself to have full control over the integration (and its parallelization)

In [ ]:
def integrand(samples):
    res = []
    for sample in samples:
        res.append(sample.c[0]**2+sample.c[1]**2)
    return res

integrator = NumericalIntegrator.continuous(2)

for i in range(10):
    samples = integrator.sample(10000 + i * 1000)
    res = integrand(samples)
    integrator.add_training_samples(samples, res)
    avg, err, chi_sq = integrator.update(1.5)
    print('Iteration {}: {:.6} +- {:.6}, chi={:.3}'.format(i + 1, avg, err, chi_sq))

Iteration 1: 0.67242 +- 0.00421607, chi=0.0
Iteration 2: 0.667966 +- 0.00197555, chi=1.43
Iteration 3: 0.666429 +- 0.00120626, chi=2.4
Iteration 4: 0.66694 +- 0.000874981, chi=2.77
Iteration 5: 0.666571 +- 0.000705219, chi=3.28
Iteration 6: 0.66681 +- 0.000600952, chi=3.7
Iteration 7: 0.666414 +- 0.000527103, chi=5.58
Iteration 8: 0.666602 +- 0.000469894, chi=6.2
Iteration 9: 0.666546 +- 0.000427166, chi=6.28
Iteration 10: 0.666811 +- 0.000392011, chi=8.73
